# Plot images + bboxes

This is similar to `sm-boox-annotation-plot.ipynb`, but with a modularized code.

In [1]:
import s3fs
import json
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict, Any

# Prefer opencv, as with PIL there's no way to load single-channel jpg as rgb in-memory data.
#from PIL import Image, ImageDraw
import cv2

# Helper functions

In [2]:
def plot_img_with_bbox(img: np.ndarray, d: Dict[Any, Any]):
    '''Draw bboxes on image'''
    for bbox in d['annotations']:
        # bbox coordinates
        x_min, y_min = bbox['left'], bbox['top']
        x_max, y_max = x_min + bbox['width'], y_min + bbox['height']

        # color to use (will round-robin r->g->b according to class_id)
        cid = bbox['class_id']
        color = [0,0,0]
        color[cid % 3] = 255
        cv2.rectangle(img, pt1=(x_min, y_min), pt2=(x_max, y_max), color=color, thickness=2)

    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.show();

In [3]:
def plot_local_img_with_bbox(json_path: str, img_root_dir: str):
    # Deserialize annotation JSON file to dictionary
    d = json.load(open(json_path, 'r'))

    # Load image into 3-channel
    img_fname = f'{img_root_dir}/{d["file"]}'
    img = cv2.imread(img_fname, 1)

    plot_img_with_bbox(img, d)

In [4]:
def plot_s3_img_with_bbox(json_path: str, img_root_dir: str, aws_profile:str='default'):
    '''
    :param aws_profile: Profile name of your AWS credential (see ~/.aws/).
                        Set to 'default' to use default credential.
    '''
    # FIXME: this is not efficient as we repeatedly re-auth to S3.
    fs = s3fs.S3FileSystem(anon=False, profile_name=aws_profile)

    # Deserialize annotation JSON file to dictionary
    d = json.loads(fs.cat(json_path))

    # Load image from S3 into 3-channel.
    # NOTE: if using bytearray(), then pair with np.asarray().
    img_fname = f'{img_root_dir}/{d["file"]}'
    b = bytes(fs.cat(img_fname))
    arr = np.frombuffer(b, dtype=np.uint8)
    img = cv2.imdecode(arr, 1)

    plot_img_with_bbox(img, d)

# Visualize squarish train image and its pieces.

## Plot an image in S3

In [ ]:
plot_s3_img_with_bbox(
    's3://bucket/data/train_annotation/image-0000001.json',
    's3://bucket/data/train/train',
    'default'
)

## Plot multiple images in local disk

In [ ]:
pieces_train = [
    '/home/ec2-user/SageMaker/train_pieces_h136_annotation/c6_img-01.json',
    '/home/ec2-user/SageMaker/train_pieces_h136_annotation/c5_img-02.json',
    '/home/ec2-user/SageMaker/train_pieces_h136_annotation/c7_img-03.json'
]
for json_path in pieces_train:
    plot_local_img_with_bbox(json_path, '/home/ec2-user/SageMaker/train_pieces_h136')